<a href="https://colab.research.google.com/github/Jollygraces/MSCRED/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pandas import concat
from pandas import DataFrame
import pandas as pd
import os
import numpy as np

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (x)
        n_out: Number of observations as output (y)
        dropnan: Boolean whether or not to drop rows with Nan values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ...t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' %(j+1, i)) for j in range(n_vars)]
    #forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
       cols.append(df.shift(-i)) 
       if i == 0:
           names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
       else:
            names += [('var%d(t+%d)' %(j+1, i)) for j in range(n_vars)]
    #put it all togehter
    agg = concat(cols, axis =1)
    agg.columns = names
    # drop rows with NAN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [ ]:
dataset= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/synthetic_data_with_anomaly_s_1.csv',header=None)

dataset=dataset.values.T

# New Section

In [ ]:
os.getcwd()

'/content'

In [ ]:
values = dataset[0:8000, :]

windowsize=5 # define your sliding window size
sensors=30 # define your number of channels/senors 

n_timepoints_to_predict=1
data=series_to_supervised(values,windowsize,n_timepoints_to_predict).reset_index(drop=True)


x_train=np.zeros((len(data),sensors,windowsize))

for i in range(len(data)):
    for j in range(windowsize):
        s_col=j*sensors
        e_col=(j+1)*sensors
        x_train[i,:,j] = data.iloc[i,s_col:e_col]
        
y_train=np.zeros((len(data),sensors,n_timepoints_to_predict))

for i in range(len(data)):
    for j in range(n_timepoints_to_predict):
        s_col=(windowsize+j)*sensors
        e_col=(windowsize+j+1)*sensors
        y_train[i,:,j] = data.iloc[i,s_col:e_col]  
        
          

In [ ]:
x_test=np.zeros((len(data),sensors,windowsize))

for i in range(len(data)):
    for j in range(windowsize):
        s_col=j*sensors
        e_col=(j+1)*sensors
        x_test[i,:,j] = data.iloc[i,s_col:e_col]
        
y_test=np.zeros((len(data),sensors,n_timepoints_to_predict))

for i in range(len(data)):
    for j in range(n_timepoints_to_predict):
        s_col=(windowsize+j)*sensors
        e_col=(windowsize+j+1)*sensors
        y_test[i,:,j] = data.iloc[i,s_col:e_col] 

In [ ]:
print('x shape {}, y shape {}'.format(x_train.shape,y_train.shape))

x shape (7995, 30, 5), y shape (7995, 30, 1)


In [ ]:
x_train[0,...]

array([[-1.196068, -1.158986, -0.813964, -0.76687 , -0.848259],
       [-1.409262, -1.022558, -0.661714, -1.225627, -1.248334],
       [-1.324885, -0.937945, -0.984585, -1.278234, -0.909862],
       [ 0.742491,  0.367879,  0.185113,  0.220149, -0.155487],
       [ 0.623979,  0.468546,  0.77372 ,  0.425162,  0.390874],
       [-0.963673, -1.126841, -1.174981, -1.385991, -1.052355],
       [-0.698226,  0.087973, -0.672968,  0.149464, -0.50563 ],
       [-0.50807 ,  0.246051,  0.410024, -0.166781, -0.481348],
       [-0.525562,  0.006991,  0.514297, -0.127469, -0.22743 ],
       [-0.858844, -0.645605, -0.15168 , -0.572383, -0.419448],
       [-1.283104, -1.369252, -1.250062, -0.895969, -0.688521],
       [-1.241669, -0.622627, -1.162868, -0.947962, -1.054963],
       [ 0.509777,  0.278355, -0.034936, -0.092271, -0.305956],
       [-0.738714, -1.209138, -0.82072 , -0.63539 , -1.105529],
       [-1.470761, -1.078936, -1.057781, -1.446965, -0.787215],
       [-1.493144, -1.06324 , -0.867468,

In [ ]:
data.iloc[0,:]

var1(t-5)   -1.196068
var2(t-5)   -1.409262
var3(t-5)   -1.324885
var4(t-5)    0.742491
var5(t-5)    0.623979
               ...   
var26(t)     0.091234
var27(t)    -0.950184
var28(t)    -0.582588
var29(t)     0.186529
var30(t)    -1.061208
Name: 0, Length: 180, dtype: float64

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)




num_heads = 5  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(sensors,windowsize))

transformer_block = TransformerBlock(windowsize, num_heads, ff_dim)
x = transformer_block(inputs)

x = layers.Dropout(0.1)(x)

outputs = layers.Conv1D(n_timepoints_to_predict,1,padding="same",activation="relu")(x)

x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(sensors, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

opt = keras.optimizers.Adam(learning_rate=0.001)


model.compile( loss="mse", metrics=["mse"], optimizer=opt)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 5)]           0         
_________________________________________________________________
transformer_block (Transform (None, 30, 5)             957       
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 5)             0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                120       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                630   

In [ ]:

history = model.fit(x_train, y_train, batch_size=32, epochs=50, validation_split=0.3)


Epoch 1/50
175/175 [==============================] - 4s 14ms/step - loss: 0.5873 - mse: 0.5873 - val_loss: 0.5953 - val_mse: 0.5953
Epoch 2/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5776 - mse: 0.5776 - val_loss: 0.5948 - val_mse: 0.5948
Epoch 3/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5772 - mse: 0.5772 - val_loss: 0.5939 - val_mse: 0.5939
Epoch 4/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5767 - mse: 0.5767 - val_loss: 0.5940 - val_mse: 0.5940
Epoch 5/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5747 - mse: 0.5747 - val_loss: 0.5938 - val_mse: 0.5938
Epoch 6/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5741 - mse: 0.5741 - val_loss: 0.5939 - val_mse: 0.5939
Epoch 7/50
175/175 [==============================] - 2s 11ms/step - loss: 0.5757 - mse: 0.5757 - val_loss: 0.5949 - val_mse: 0.5949
Epoch 8/50
175/175 [==============================] - 2s 11ms/step - 

In [ ]:
n_timepoints_to_predict

1

In [ ]:
# Evaluate the model on the test data using 'evaluate'
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (Probabilities-- the output of the last layer)
#on new data using 'predict'
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predicttions shape:", predictions.shape)

Evaluate on test data


NameError: ignored

In [ ]:
data.head()

,var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var7(t-5),var8(t-5),var9(t-5),var10(t-5),var11(t-5),var12(t-5),var13(t-5),var14(t-5),var15(t-5),var16(t-5),var17(t-5),var18(t-5),var19(t-5),var20(t-5),var21(t-5),var22(t-5),var23(t-5),var24(t-5),var25(t-5),var26(t-5),var27(t-5),var28(t-5),var29(t-5),var30(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var8(t-4),var9(t-4),var10(t-4),...,var21(t-1),var22(t-1),var23(t-1),var24(t-1),var25(t-1),var26(t-1),var27(t-1),var28(t-1),var29(t-1),var30(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t),var13(t),var14(t),var15(t),var16(t),var17(t),var18(t),var19(t),var20(t),var21(t),var22(t),var23(t),var24(t),var25(t),var26(t),var27(t),var28(t),var29(t),var30(t)
0,0.762990,0.406899,0.297291,0.525051,0.978470,0.201931,0.290573,0.613655,-1.211027,-0.635631,0.056862,-0.382054,-0.225819,-1.134494,-0.870654,-1.010661,-0.793196,0.869693,-1.094720,0.490602,-0.990073,-1.050471,0.563175,0.606927,0.736851,-0.904726,1.061046,-0.710372,0.401555,0.126757,0.568047,0.444322,0.022958,1.090986,0.566341,-0.101490,0.942926,0.537513,-0.726271,-0.110011,...,-1.120967,-1.185134,-0.470560,0.734541,1.034534,-0.936044,0.622915,-1.084722,0.795337,0.607879,0.306990,0.373717,0.371918,0.825970,0.865167,0.230210,0.745923,0.226774,-0.789510,-0.932413,-0.095239,-0.961368,-0.281853,-0.686418,-0.852382,-1.088180,-0.706236,0.073249,-0.331958,0.318147,-0.657651,-0.929257,-0.125569,0.431442,0.736264,-0.675956,1.057765,-0.891272,0.824883,0.896375
1,0.568047,0.444322,0.022958,1.090986,0.566341,-0.101490,0.942926,0.537513,-0.726271,-0.110011,-0.419921,-0.473355,0.114155,-0.810490,-0.987165,-0.609554,-0.574198,0.607861,-1.214328,-0.062996,-1.431003,-1.052125,-0.197734,1.342867,0.796833,-0.437865,0.764878,-1.116260,1.349490,0.857254,0.415850,-0.018484,0.243283,1.175204,0.363261,0.141624,0.440737,0.492541,-0.735914,-0.826918,...,-0.657651,-0.929257,-0.125569,0.431442,0.736264,-0.675956,1.057765,-0.891272,0.824883,0.896375,0.571870,-0.367850,0.240408,0.758343,0.745164,0.290634,0.308606,0.386941,-1.213793,-0.355800,0.140189,-0.248923,0.327850,-0.932592,-0.728535,-0.994416,-0.707106,-0.060294,-0.820859,0.257585,-0.374587,-1.428231,0.005733,0.417051,0.521078,-0.480877,0.765196,-0.907137,0.685927,1.192263
2,0.415850,-0.018484,0.243283,1.175204,0.363261,0.141624,0.440737,0.492541,-0.735914,-0.826918,-0.025770,-0.744635,-0.026306,-1.775526,-0.489343,-0.796044,-1.204955,0.461865,-0.574486,0.839972,-1.062310,-0.904209,-0.579833,0.490907,1.025536,-0.621935,0.531138,-0.561322,0.492599,1.352448,1.081596,-0.352860,0.300445,1.298145,0.562864,0.005497,0.890673,-0.376789,-0.816013,-1.013894,...,-0.374587,-1.428231,0.005733,0.417051,0.521078,-0.480877,0.765196,-0.907137,0.685927,1.192263,1.031251,0.058009,0.264507,0.150668,0.641451,0.299449,0.172414,-0.206491,-0.874928,-0.480246,-0.101200,-0.812941,0.132650,-0.584935,-0.747685,-0.854332,-0.886898,0.430033,-0.905311,0.406291,-1.194963,-1.219262,-0.531110,1.243236,1.480625,-1.143109,0.583059,-0.625422,0.954316,0.646285
3,1.081596,-0.352860,0.300445,1.298145,0.562864,0.005497,0.890673,-0.376789,-0.816013,-1.013894,-0.061683,-0.360432,-0.207074,-1.070409,-0.888810,-0.776957,-0.890599,0.139792,-1.149126,-0.174026,-1.423350,-0.912458,-0.186518,0.421492,0.728799,-0.907804,1.711936,-1.434381,0.432975,1.338966,1.205209,0.015392,0.762171,0.930388,0.748725,0.219969,0.580964,0.022865,-1.508609,-1.291141,...,-1.194963,-1.219262,-0.531110,1.243236,1.480625,-1.143109,0.583059,-0.625422,0.954316,0.646285,0.742341,-0.153773,0.733898,0.452351,1.028769,0.300091,0.032161,-0.063109,-1.546215,-0.779246,0.438107,-0.505577,-0.167850,-1.063830,-0.541746,-0.511883,-0.332145,0.503457,-1.162375,0.634318,-1.059358,-0.404213,-0.805195,0.004624,0.646308,-0.700481,1.330816,-1.348242,0.605999,0.954864
4,1.205209,0.015392,0.762171,0.930388,0.748725,0.219969,0.580964,0.022865,-1.508609,-1.291141,0.036676,-0.633501,0.271377,-0.868951,-1.132427,-0

In [ ]:
history.history

{'loss': [0.5848652720451355,
  0.5776777863502502,
  0.5770824551582336,
  0.5767571926116943,
  0.5765029788017273,
  0.5761932730674744,
  0.575915515422821,
  0.575783908367157,
  0.5755518078804016,
  0.5755314826965332,
  0.5755214095115662,
  0.5752689242362976,
  0.5754208564758301,
  0.5751962065696716,
  0.5751596689224243,
  0.5750212669372559,
  0.5748469233512878,
  0.5747756361961365,
  0.574835479259491,
  0.5749061703681946],
 'mse': [0.5848652720451355,
  0.5776777863502502,
  0.5770823955535889,
  0.5767571926116943,
  0.5765029788017273,
  0.5761932730674744,
  0.575915515422821,
  0.575783908367157,
  0.5755518078804016,
  0.5755314826965332,
  0.5755214095115662,
  0.5752689242362976,
  0.5754208564758301,
  0.5751962065696716,
  0.5751596689224243,
  0.5750212669372559,
  0.5748469233512878,
  0.5747756361961365,
  0.574835479259491,
  0.5749061703681946],
 'val_loss': [0.5940602421760559,
  0.5939434766769409,
  0.5936709046363831,
  0.5943921208381653,
  0.59438

In [ ]:
values = dataset[10001:20000, :]

windowsize=5 # define your sliding window size
sensors=30 # define your number of channels/senors 

n_timepoints_to_predict=1
data=series_to_supervised(values,windowsize,n_timepoints_to_predict).reset_index(drop=True)


x_test=np.zeros((len(data),sensors,windowsize))

for i in range(len(data)):
    for j in range(windowsize):
        s_col=j*sensors
        e_col=(j+1)*sensors
        x_test[i,:,j] = data.iloc[i,s_col:e_col]
        
y_test=np.zeros((len(data),sensors,n_timepoints_to_predict))

for i in range(len(data)):
    for j in range(n_timepoints_to_predict):
        s_col=(windowsize+j)*sensors
        e_col=(windowsize+j+1)*sensors
        y_test[i,:,j] = data.iloc[i,s_col:e_col]    

In [ ]:
# Evaluate the model on the test data using 'evaluate'
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (Probabilities-- the output of the last layer)
#on new data using 'predict'
print("Generate predictions for 2 samples")
predictions = model.predict(x_test[:2])
print("predicttions shape:", predictions.shape)

Evaluate on test data
313/313 [==============================] - 1s 4ms/step - loss: 0.5807 - mse: 0.5807
test loss, test acc: [0.5807207226753235, 0.5807207226753235]
Generate predictions for 2 samples
predicttions shape: (2, 30)


In [ ]:
model.fit(x_test, y_test, batch_size=32, epochs=50, validation_split=0.3)


Epoch 1/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5785 - mse: 0.5785 - val_loss: 0.5946 - val_mse: 0.5946
Epoch 2/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5783 - mse: 0.5783 - val_loss: 0.5947 - val_mse: 0.5947
Epoch 3/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5782 - mse: 0.5782 - val_loss: 0.5942 - val_mse: 0.5942
Epoch 4/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5780 - mse: 0.5780 - val_loss: 0.5941 - val_mse: 0.5941
Epoch 5/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5779 - mse: 0.5779 - val_loss: 0.5941 - val_mse: 0.5941
Epoch 6/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5776 - mse: 0.5776 - val_loss: 0.5945 - val_mse: 0.5945
Epoch 7/50
219/219 [==============================] - 2s 11ms/step - loss: 0.5776 - mse: 0.5776 - val_loss: 0.5942 - val_mse: 0.5942
Epoch 8/50
219/219 [==============================] - 2s 11ms/step - 

# New Section

# New Section